In [1]:
import shelve

time = 11

f = shelve.open("washedPoints-Closer-Multi3-%d.dat"%(time-1))

print f.keys()

['Bound_Points', 'Inner_Points']


In [2]:
from pprint import pprint
import handytools as hd

bound_Points = hd.mylst(f["Bound_Points"])
pprint(bound_Points[:20:2])

[array([ 58.        ,  -6.32033443,   4.89626932], dtype=float32),
 array([ 56.        ,  -8.97367954,   4.85090446], dtype=float32),
 array([ 54.        , -11.11724663,   4.78782272], dtype=float32),
 array([ 52.        , -12.88874817,   4.80532455], dtype=float32),
 array([ 50.        , -14.40356255,   4.80340052], dtype=float32),
 array([ 48.        , -15.72612953,   4.80083132], dtype=float32),
 array([ 46.        , -16.84690857,   4.79559231], dtype=float32),
 array([ 44.        , -17.72258568,   4.77669334], dtype=float32),
 array([ 42.        , -18.24643135,   4.73627806], dtype=float32),
 array([ 40.        , -18.33431244,   4.69622374], dtype=float32)]


In [3]:
from operator import itemgetter
import numpy as np

avg =  sum(bound_Points.getz()) / len(bound_Points)
stderr = np.std(bound_Points.getz(),)
print "The average is: %.3f, the std error is :%.3f"%(avg, stderr)

alpha_1 = 0.5
alpha_2 = 1.0

new_points = hd.mylst([(1.375, 0.0), (60.0, 0.0)])

def adjustPoint(x_val, y_val, z_val, z_avg, alpha_low=0.5, alpha_high=1.0):
    if z_val < z_avg:
        y_adjusted = y_val + alpha_low*(z_val - z_avg)
        point = (x_val, max(0.6, y_adjusted))
    else:
        y_adjusted = y_val + alpha_high*(z_val - z_avg)
        point = (x_val, max(0.6, y_adjusted))
    return point

for x, y, z in bound_Points:
    # 利用 y > 0 把原来的端点也过滤掉
    if x > 0 and y > 0:
        point = adjustPoint(x, y, z, avg, alpha_low=0.5, alpha_high=0.5)
        if abs(point[0] - 1.0) < 0.001 and point[1] < 1.05:
            point = (1.0, 1.4)
        new_points.append(point)

new_points = hd.mylst(sorted(new_points, key=itemgetter(0),))

pprint(new_points)

The average is: 4.832, the std error is :0.149
[(1.375, 0.0),
 (1.4847169, 1.086465897058186),
 (1.729423, 2.0909269470917553),
 (2.0, 2.9486175198304028),
 (2.0795135, 3.083604397271809),
 (2.5759869, 4.0931826252686356),
 (3.0, 4.939295114968953),
 (3.0981302, 5.0791288037049149),
 (3.7382767, 6.0787592549073075),
 (4.0, 6.4817371983277177),
 (4.4266548, 7.0718389172303056),
 (5.0, 7.8457115311371659),
 (5.1826763, 8.0612192768799638),
 (6.0, 9.0242181439148759),
 (6.0295858, 9.0486829895722245),
 (6.988143, 10.036239447091756),
 (7.0, 10.048327984307942),
 (8.0, 10.977765621637044),
 (8.0722322, 11.036993326638875),
 (9.0, 11.80338651130074),
 (9.2987232, 12.031843246911702),
 (10.0, 12.552627863382039),
 (10.6806, 13.025325598214803),
 (11.0, 13.241516174768147),
 (12.0, 13.884188713525472),
 (12.217701, 14.020612539743123),
 (13.0, 14.458967031930623),
 (14.0, 14.987358870004353),
 (14.017414, 15.017646135781941),
 (15.0, 15.450690330957112),
 (16.0, 15.856473268960652),
 (16.3540

In [4]:
import matplotlib.pyplot as plt

plt.figure()
plt.scatter(new_points.getx(),new_points.gety())
plt.show()

In [5]:
result = hd.coord_generator(new_points,interval=1.0, start=1.0, end=60.0)
print 'the keys of output is:\n\t',result.keys()

The interpolation points sorted by x:
	[(1.375, 0.0), (1.4847169, 1.086465897058186), (1.729423, 2.0909269470917553), (2.0, 2.9486175198304028), (2.0795135, 3.083604397271809), (2.5759869, 4.0931826252686356), (3.0, 4.939295114968953), (3.0981302, 5.0791288037049149), (3.7382767, 6.0787592549073075), (4.0, 6.4817371983277177), (4.4266548, 7.0718389172303056), (5.0, 7.8457115311371659), (5.1826763, 8.0612192768799638), (6.0, 9.0242181439148759), (6.0295858, 9.0486829895722245), (6.988143, 10.036239447091756), (7.0, 10.048327984307942), (8.0, 10.977765621637044), (8.0722322, 11.036993326638875), (9.0, 11.80338651130074), (9.2987232, 12.031843246911702), (10.0, 12.552627863382039), (10.6806, 13.025325598214803), (11.0, 13.241516174768147), (12.0, 13.884188713525472), (12.217701, 14.020612539743123), (13.0, 14.458967031930623), (14.0, 14.987358870004353), (14.017414, 15.017646135781941), (15.0, 15.450690330957112), (16.0, 15.856473268960652), (16.354059, 15.998920263742146), (17.0, 16.1775

In [6]:
hd.result_visualize(result)
hd.abaqusInputDataGenerator_shelveVersion(outPath="modelPoints-Current.dat", time=time, **result)

shelveVersion Complete!


In [7]:
# a = max(-0.66,0.1)

# print a